# Earthquake risk based on location in Japan

**จัดทำโดย   นายเจตพล  เซาะจอหอ    1600900730
            นายชยุต   สมวงษ์อินทร์   1600900987


## Workflow stages


1. Question or problem definition.
2. Acquire training and testing data.
3. Wrangle, prepare, cleanse the data.
4. Analyze, identify patterns, and explore the data.
5. Model, predict and solve the problem.
6. Visualize, report, and present the problem solving steps and final solution.



## Question and problem definition

เนื่องจากประเทศญี่ปุ่นเกิดแผ่นดินไหวบ่อย แต่หากเราจำเป็นต้องอาศัยอยู่ในญี่ปุ่นอาจจะต้องมีข้อมูลของแต่ละเมืองหรือจุดที่จะอยู่ว่ามีแผ่นดินไหวมากน้อยรุนแรงแค่ไหน
จึงจะสร้าง Model ที่สามารถบอกได้ว่าพิกัดนี้อยู่ในโซนความรุนแรงระดับใด

In [ ]:
### เรียกใช้ Library และ Load data

import numpy as np
import pandas as pd 
import datetime

# Visualization
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
%matplotlib inline
import folium
from folium import *
from folium.plugins import *
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import pandas as pd
import os
from subprocess import check_output

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import geopandas as gpd

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.model_selection import train_test_split  
from sklearn.metrics import confusion_matrix

from pathlib import Path

train_data = pd.read_csv("../input/earthquakes-in-japan/Japan earthquakes 2001 - 2018.csv")

# Analyze by describing data

 ดูตัวแปรว่ามีอะไรบ้าง

In [ ]:
print(train_data.columns.values) #

* Latitude : มุมที่วัดได้จากแนวเส้นขนานพื้น โดยกำหนดให้ค่าเส้นศูนย์สูตรโลกเป็นศูนย์
* Longitude : เส้นแกนศูนย์ของลองจิจูด ทำให้เกิดการแบ่งโซนภูมิศาสตร์โลกเป็นทวีปต่าง ๆ
* Time : วันที่และเวลา
* Depth : ความลึกของจุดศูนย์กลางแผ่นดินไหวจากผิวโลก
* Mag : ระดับความสั่นสะเทือน
* Place : คำอธิบายข้อความของพื้นที่ทางภูมิศาสตร์ที่ตั้งชื่อใกล้กับเหตุการณ์
* MagType : หน่วยของ Magnitude
* Nst : จำนวนของสถานีที่ใช้วัดค่าแผ่นดินไหว
* Gap : ตำแหน่งมุมazimuthalที่ใหญ่ที่สุดระหว่างสถานีที่อยู่ติดกัน [0.0,180.0]
* Dmin : ระยะทางแนวนอนจากศูนย์กลางของแผ่นดินไหวไปยังสถานีที่ใกล้ที่สุด [0.4,7.1]
* Rms : เวลาในการเดินทางของค่าเฉลี่ยราก(RMS)เป็นวินาทีโดยใช้น้ำหนักทั้งหมด
* Net : ID ของผู้สนับสนุนข้อมูล.
* ID : ตัวบ่งชี้หรือตัวแปรเฉพาะสำหรับเหตุการณ์นั้นๆ
* Updated : เวลาที่เหตุการณ์ได้รับการอัปเดตล่าสุด 
* Type : ประเภทของเหตุการณ์แผ่นดินไหว
* HorizontalError : ความไม่แน่นอนของตำแหน่งที่รายงานของเหตุการณ์เป็นกิโลเมตร
* DepthError : ความไม่แน่นอนของรายงานความลึกของเหตุการณ์เป็นกิโลเมตร
* MagError : ความไม่แน่นอนของความแรงที่เกิดขึ้นของเหตุการณ์
* Magnst : จำนวนสถานีแผ่นดินไหวที่ใช้ในการคำนวณขนาดของแผ่นดินไหวครั้งนี้
* Status : ตัวแปรว่าเหตุการณ์ได้รับการตรวจสอบโดยมนุษย์
* LocationSource : ตำแหน่งเครือข่ายที่เขียนรายงานสถานที่ของเหตุการณ์นี้
* MagSource : ตำแหน่งเครือข่ายที่เขียนรายงานความแรงของเหตุการณ์นี้

In [ ]:
train_data.head() #ดูข้อมูลในตาราง

In [ ]:
train_data.shape

ดูขนาดของข้อมูลมีทั้งหมด 22 column และ 14092 แถว

ดูข้อมูลส่วนท้ายตาราง

In [ ]:
train_data.tail()

### Cleans the data.

#### เลือก column ข้อมูลที่มีประโยชน์

In [ ]:
train_data = train_data[['time','latitude', 'longitude', 'depth','mag','place']]

คัดเลือก Column 6 ตัวนี้เพราะตัวแปรอื่นๆใน Dataset เป็นค่าที่ไม่ได้ใช้งานและเป็นแค่ส่วนกำกับข้อมูลเท่านั้นเช่นค่า Spec ของเครื่องที่ใช้วัดสถานที่วัดและความคลาดเคลื่อน

In [ ]:
NAs = pd.concat([train_data.isnull().sum()], axis=1, keys=['null'])
NAs[NAs.sum(axis=1) > 0]

ทำการเช็คข้อมูลใน column ที่ตัดแล้วไม่มีค่า null

ทำ Feature เพื่อแยกชื่อเมืองและประเทศออกจาก Place จาก data หลาย format

In [ ]:
train_data[['temp', 'country']] = train_data['place'].str.split(',\s+', expand=True)

train_data[['country','place']].groupby(['country']).count().rename(columns={"place":"count"})

จากข้อมีตำแหน่งของประเทศอื่นปนมาด้วยจะทำการตัดออกเนื่องจากเราสนใจแต่ประเทศญี่ปุ่น และ ประเทศญี่ปุ่นมี 2 format คือ Japan และ Japan region จะทำการยุบเป็นอันเดียว

In [ ]:
train_data = train_data[~train_data['country'].isin(['China','North Korea','Russia','South Korea'])]
train_data.country.fillna(value=pd.np.nan, inplace=True)
train_data['country'] = train_data['country'].replace('Japan region', 'Japan')
train_data['country'] = train_data['country'].replace(pd.np.nan, 'Japan')
train_data[['country','place']].groupby(['country']).count().rename(columns={"place":"count"})

ได้ข้อมูลที่ Filter แล้วทั้งหมดเป็น Japan ทั้งหมด โดยการใช้คำสั่ง split จากตัวอักษร , ทำให้บาง field ที่ไม่มี , ข้อมูลเป็น Null ซึ่งได้ทำการเช็คแล้ว Japan ตกไปอยู่ใน col temp จึงเรียกมาทับ col country ทำให้ได้ข้อมูลทั้งหมด 13445 แถว

### ดูค่าเฉลี่ย,ส่วนเบี่ยงแบน ในข้อมูล

In [ ]:
train_data.describe()

* ความรุนแรงของแผ่นดินไหว อยู่ที่ 4.8 ริกเตอร์
* ค่า [mag] std น้อย 0.37 ข้อมูลไม่กระจาย
* IQR อยู่ที่ 4.6-4.9 ค่อนข้างแคบ
* ค่าเฉลี่ยความลึกของศูนย์กลางแผ่นดินไหว 51.36 km.
* ค่าส่วนเบี่ยงเบนมาตราฐานความลึกของศูนย์กลางแผ่นดินไหว = 76.603 km.
* พิกัดศูนย์กลางแผ่นดินไหวจากค่าเฉลี่ย latitude 37.410294 , longitude 142.980441

### แปลงค่าวันเวลาให้เป็น UNIX
##### เพื่อให้ง่ายต่อการวิเคราะห์ในขั้นตอนการสร้าง Model (Just in case)

In [ ]:
#จัด format 2001-01-04T04:18:21.430Z ให้เป็น 2001-01-04 04:18:21.430000+00:00
train_data['time'] = pd.to_datetime(train_data['time'],format= '%Y-%m-%dT%H:%M:%S') 
#แปลงเวลาเป็น UNIX Time Stamp เพื่อให้ง่ายต่อการจัดการ

train_data['timestamp'] = train_data.time.values.astype(np.int64) 

train_data.head()

# Analyze by pivoting features

เพิ่มค่า วัน-เดือน-ปี จาก column time

In [ ]:
#หาวัน(ตัวเลข)และเพิ่มใน col
train_data['day_of_week'] = pd.to_datetime(train_data['time']).dt.weekday+1
#หาเดือนและเพิ่มใน col
train_data['month'] = pd.to_datetime(train_data['time']).dt.month
#หาปีและเพิ่มใน col
train_data['year'] = pd.to_datetime(train_data['time']).dt.year

### จากข้อมูลที่มีแยกประเภท Data ได้ดังนี้ ### 

**ประเภทของ Column ที่เป็นตัวเลข Categorical**
* Categorical: day_of_week,day_of_week_name,month,year
****[](http://)
**ประเภทของ Column ที่เป็นตัวเลข**
* Continous:  time,latitude,longitude,depth,mag.

### หาข้อมูลว่าในแต่ละเดือนในแต่ละปีมีแผ่นดินไหวบ่อยขนาดไหน ### 
จากข้อมูลที่มีจะเลือกการแสดงผลแบบ Heatmap โดยให้แกนนอนเป็นปี และ แกนตั้งเป็นเดือน

In [ ]:
## ทำ Pivot จากปีและเดือนโดยนับค่าครั้งที่เกิด
pt = train_data.pivot_table(index="month",columns="year",values="mag", aggfunc="count") 
#กำหนดขนาด และสี Heatmap 
fig, ax = plt.subplots(figsize=(8,6)) 
#แสดง Heatmap
sns.heatmap(pt,cmap="Blues",annot=True,fmt='g')

จาก Heatmap แสดงให้เห็นว่าในปี 2011 เดือน 3 นั้นมีจำนวนครั้งของแผ่นดินไหนที่เยอะมากถึง 1984 ครั้งและในเดือนอื่นๆของปีนั้นก็มีค่ามาก จึงทำการตัดข้อมูลของเดือน 3 ปี 2011 ออก

In [ ]:
### เอาข้อมูลปี 2011 เดือน 3 ออก
## ทำ Pivot และ Plot อีกครั้ง
## ทำ Pivot จากปีและเดือนโดยนับค่าครั้งที่เกิด

indexNames = train_data[ (train_data['month'] == 3) & (train_data['year'] == 2011) ].index
train_data.drop(indexNames , inplace=True)

pt = train_data.pivot_table(index="month",columns="year",values="mag", aggfunc="count") 
#กำหนดขนาด และสี Heatmap 
fig, ax = plt.subplots(figsize=(8,6)) 
#แสดง Heatmap
sns.heatmap(pt,cmap="Blues",annot=True,fmt='g')

จากการตัดเดือน 3 ปี 2011 ออก Data โดยรวมดูใกล้เคียงกัน

### หาว่าแต่ละวันวันไหนมีโอกาสเกิดแผ่นดินไหวเยอะสุด ###

In [ ]:
#ทำการนับแถวที่เกิดขึ้นในแต่ละวันแล้วนำค่าที่นับได้ในแต่ละวันมาคิดเป็น % ของทั้ง 7 วัน
train_data[['day_of_week','mag']].groupby(['day_of_week']).count().rename(columns={"mag":"% of count"}).apply(lambda x: 100 * x / float(x.sum())).round(2).rename(index={
    1:"Monday",2:"Tuesday",3:"Wednesday",4:"Thursday",5:"Friday",6:"Saturday",7:"Sunday"
})

จากข้อมูลจะเห็นได้ว่าแต่ละ วันในสัปดาห์มีโอกาสเกิดแผ่นดินไหวใกล้เคียงกัน

### หาว่าแต่ละเดือน เดือนไหนมีโอกาสเกิดแผ่นดินไหวเยอะสุด

In [ ]:
#ทำการนับแถวที่เกิดขึ้นในแต่ละเดือนแล้วนำค่าที่นับได้ในแต่ละเดือนมาคิดเป็น % ของทั้ง 7 วัน
train_data[['month','mag']].groupby(['month']).count().rename(columns={"mag":"% of count"}).apply(lambda x: 100 * x / float(x.sum())).round(2).rename(index={
    1:"January",2:"February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"
})

จากตารางวิเคราะห์หาการเกิดแผ่นดินไหวตามเดือนที่เกิดขึ้น พบว่ามีการเกิดแผ่นดินไหวใกล้เคียงกันและช่วงท้ายปีเดือน 11-12 มีค่าเยอะกว่าช่วงกลางปีเท่าตัว แต่เมื่อย้อนไปดูข้อมูลภาพรวมแต่ละปีด้านบนข้อมูลเพิ่มมากจากเดือน 11/2006 , 12/2010 ,12/2012 แสดงว่าโอกาศเกิดในแต่ละเดือนเท่ากัน

### **หาว่าแผ่นดินไหวกลางวันหรือกลางคืนมากกว่ากัน** ###

In [ ]:
## ทำ feature แบ่งช่วงเวลาชั่วโมงเป็น 6 ช่วงเวลา ให้ช่วงเวลา 0-4 เป็น LateNight 4-8 เป็น Early Morning เป็นต้นแล้วเพิ่มใน Column session
b = [0,4,8,12,16,20,24]
l = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
train_data['session'] = pd.cut(pd.to_datetime(train_data['time']).dt.hour, bins=b, labels=l).replace(np.nan, 'Night') ##np.nan คือเวลาชั่ว 0 เป็น NaN จึงแทนด้วย Night 
## ทำ Pivot เพื่อดูว่าเวลาแต่ละช่วงเกิดแผ่นดินไหวเท่าไรโดยนับจำนวนครั้งที่เกิดในแต่ละเวลาแล้วนำมาคิดว่าช่วงเวลานั้นเป็นกี่ % เมื่อเทียบกับช่วงเวลาอื่นทั้งหมด
train_data[['session','mag']].groupby(['session']).count().rename(columns={"mag":"% of count"}).apply(lambda x: 100 * x / float(x.sum())).round(2)

จากการรันแต่ละช่วงเวลาที่เกิดแผ่นดินไหวนั้นมีอัตราส่วนที่เท่าๆ

### สรุปในส่วนนี้จากข้อมูลที่ Explore การเกิดแผ่นดินไหวนั้นเป็นสิ่งที่ไม่แน่นอนมีโอกาสเกิดขึ้นได้ตลอดไม่มี Pattern

# Analyze by Visualizing data

หาว่าเมืองไหนแผ่นดินไหวสูงสุด

In [ ]:
train_data['temp'] = train_data['temp'].str.replace("the","")
train_data[['temp1','temp2']] = train_data['temp'].str.split(' of ', expand = True)
train_data.temp2.fillna(value=pd.np.nan, inplace=True)
#pd.set_option('display.max_rows', 1000)
train_data.loc[train_data["temp2"].isnull(),'temp2'] = train_data["temp1"];
train_data=train_data.rename(columns={"temp2":"city"})
train_data = train_data[['time','latitude', 'longitude', 'depth', 'mag','city']]
train_data = train_data[~train_data['city'].isin(['China-Russia-North Korea border region','North Korea','South Korea','Taiwan','Japan','eastern Russia-norastern China border region'])] #ลบแถวที่ชื่อCityname ไม่เกี่ยวกับญี่ปุ่น
train_data['city'] = train_data['city'].replace('eastern Honshu', 'Honshu')
train_data['city'] = train_data['city'].replace('western Honshu', 'Honshu')
train_data['city'] = train_data['city'].replace('southwestern Ryukyu Islands', 'Ryukyu Islands')
display_grouped = train_data[['city','mag']].groupby(['city']).count().rename(columns={"mag":"count"})
display_grouped.sort_values('count', ascending=False)

ทำการ Clean ให้เหลือเฉพาะ Cityname แล้วทำการเรียงผลที่ได้ Honshu มีจำนวนแผ่นดินไหวเยอะที่สุดจึงได้ทำการค้นหาใน Google ว่าอยู่ตำแหน่งได้คำตอบที่ได้คือ Honshu ชื่อของเกาะญี่ปุ่นที่เป็นเกาะหลักจึงได้ทำการเช็ค Data เต็มก่อนตัดคำ (150KM West of Honshu) สรุปได้ว่าเป็นตำแหน่งที่เกิดขึ้นในทะเล

In [ ]:
mpl.rcParams['figure.figsize']=(16.0,10.0)    #(6.0,4.0)
mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=1900             #72 
mpl.rcParams['figure.subplot.bottom']=.1 

word_string=' '.join(train_data['city'].str.lower())
STOPWORDS = {'islands','islands','honshu','honshu','eastern','eastern','western','western'}
wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white', 
                      max_words=100,random_state = 42,collocations=False
                         ).generate_from_text(word_string)

plt.clf()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

Visualization ของเมืองที่เกิดแผ่นดินไหวบ่อยโดยได้ทำการตัดคำว่า honshu ออกและ eastern western ออกเนื่องจาก WordCloud คำนวนค่าจาก string ที่ติดกันเท่านั้น จากภาพจุดที่เกิดแผ่นดินไหวบ่อยจะตัวหนังสือชื่อเมืองตัวใหญ่

### Heatmap ของจุดที่เกิดแผ่นดินไหว

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap

map_hooray = folium.Map(location=[38.495035, 137.026582],
                    zoom_start = 4) 

# Ensure you're handing it floats
#xx = train_data.loc[train_data['year'] == 2012]
xx = train_data
xx['latitude'] = xx['latitude'].astype(float)
xx['longitude'] = xx['longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs

xx = xx[['latitude', 'longitude']]
xx = xx.dropna(axis=0, subset=['latitude','longitude'])

# List comprehension to make out list of lists
xx = [[row['latitude'],row['longitude']] for index, row in xx.iterrows()]

# Plot it on the map
HeatMap(xx,min_opacity=0.4,radius=16, blur=10).add_to(map_hooray)

# Display the map
map_hooray

จากแผนภาพตำแหน่งที่เกิดแผ่นดินไหวนำมา plot บน HeatMap เมื่อซูมเข้าไปจุดสีแดงคือบริเวณนั้นเกิดแผ่นดินไหวหลายครั้ง โดยส่วนมากเกิดขึ้นในทะเลมากกว่าบนบก ตำแหน่งที่เกิดบนบกเยอะคือเมือง tokyo และเมืองทางตอนใต้ของแผ่นดิน รูปทรงตำแหน่งมีแนวของจุกที่เกิดแผ่นดินไหวอยู่ตามแนวเกาะเล็กๆในทะเล และตำแหน่งแผ่นดินไหวจะเกิดในตะวันออกของญี่ปุ่นมากกว่า

### Chart ของความลึกของแผ่นดินไหวว่าแต่ละระดับมีเท่าใดบ้าง

In [ ]:
mgchart = train_data[['latitude','mag']].groupby(['mag']).count().rename(columns={"latitude":"count"})
sns.distplot(train_data['mag'], hist=True, kde=True, 
             bins=40, color = 'blue',
             hist_kws={'edgecolor':'black'})
mgchart.head(15)

### Chart ของความแรงแผ่นดินไหวว่าแต่ละระดับเกิดขึ้นมากน้อยเพียงใด

In [ ]:
mgchart2 = train_data[['latitude','depth']].groupby(['depth']).count().rename(columns={"latitude":"count"})
sns.distplot(train_data['depth'], hist=True, kde=True, 
             bins=40, color = 'blue',
             hist_kws={'edgecolor':'black'})
mgchart2.head()

ความแรงที่เกิดขึ้นส่วนมากอยู่ที่ระดับ 4.5-5 ริกเตอร์ เราอาจจะต้องแบ่งประเภทของความรุนแรง(mag)เป็นช่วงแล้วนำไปทำ categorize เพื่อไปใช้ใน Model
* 4.0 - 4.9 แผ่นดินไหวขนาดค่อนข้างเล็ก (Light)    = 1
* 5.0 - 5.9 แผ่นดินไหวขนาดปานกลาง (Moderate)  = 2
* 6.0 - 6.9 แผ่นดินไหวขนาดค่อนข้างใหญ่ (Strong)  = 3
* 7.0 - 7.9 แผ่นดินไหวขนาดใหญ่ (Major)         = 4
* มากกว่า 8.0 แผ่นดินไหวใหญ่มาก (Great)          = 5

In [ ]:
# ทำ feature โดยใช้ pd.cut แบ่งช่วงตามด้านบน
bins = [4, 5, 6, 7, 8, np.inf]
names = ['1', '2', '3', '4', '5']
train_data['mag_cat'] = pd.cut(train_data['mag'], bins, labels=names)

### หาจำนวนครั้งที่เกิดในแต่ละระดับ

In [ ]:
train_data[['mag_cat','city']].groupby(['mag_cat']).count().rename(columns={"mag":"% of count"}).rename(columns={"city":"count"})

จำนวนครั้งที่เกิดแผ่นดินไหวในความรุนแรงแต่ละระดับจะเห็นได้ว่ามีแผ่นดินไหวที่มีความรุนแรงมาเกิดขึ้นน้อยครั้ง

### นำความรุนแรงแต่ละระดับมาดูใน Heatmap ว่าจุดที่ความรุนแรงมากอยู่ตำแหน่งไหน

In [ ]:
map_hooray2 = folium.Map(location=[38.495035, 137.026582],
                    zoom_start = 4) 
## ทดลองเลือกความรุนแรงระดับ 4 และ 5
yy = train_data.loc[train_data['mag_cat'].isin(['4','5'])]
yy['latitude'] = yy['latitude'].astype(float)
yy['longitude'] = yy['longitude'].astype(float)

yy = yy[['latitude', 'longitude']]
yy = yy.dropna(axis=0, subset=['latitude','longitude'])

yy = [[row['latitude'],row['longitude']] for index, row in yy.iterrows()]

HeatMap(yy,min_opacity=0.8,radius=20, blur=10).add_to(map_hooray2)

map_hooray2

จาก heatmap คือจุดที่เกิดแผ่นดินไหว cat 4 และ 5 มีบางจุดสีแดงเป็นจุดที่เกิดขึ้นซ้ำดังนั้นพื้นที่ๆใกล้เคียงจุดที่รุนแรงจะเป็นอันตราย จากตรงนี้จึงนำ Lat/Long มาใช้เป็นเป็นตัวแปรในการทำ Model เพื่อหาว่าจุดใดเสี่ยงมากน้อยเพียงใด

# Model 
จากการหาข้อมูลต่างๆด้านบนจึงเลือกออกแบบ Model การคาดเดาความรุนแรงของแผ่นดินไหวที่อาจจะเกิดขึ้นในพิกัดนั้นๆ โดยใช้ข้อมูล longitude latitude และ ชื่อเมือง (city) เพื่อให้ผู้ที่ต้องการอยู่อาศัยได้เช็คว่าตัวเองอยู่ในพื้นที่ความเสี่ยงระดับใด [mag_cat] รูปแบบการทำ ML แบบ supervised จากข้อมูลเลือกใช้ Model แบบ Classification เนื่องจากค่าที่ต้องการให้ Model Predict ออกมาเป็นระดับความรุนแรงที่แบ่งไว้ 5 ระดับ 1-5

## ขั้นตอน 
1. นำข้อมูล city มาทำเป็น one hot
2. แบ่ง Test และ Train Data
3. เริ่มจากการ Train Model หลายๆแบบ
4. หาค่า Performane 
5. เปรียบเทียบ Performace แต่ละแบบ
6. เลือก Model ว่าจะใช้ Model แบบไหนในการใช้งาน
7. นำ Model ไปใช้งาน

### แบ่ง Train_data และ Test_data

นำ city มาทำเป็น one hot เนื่องจากข้อมูลเป็น categorical ไม่สามารถรันใน Model ได้และ Join กับ lat,long จาก data หลัก และกำหนดเป็นตัวแปร X
ส่วนตัวแปร y จะเป็น column mag_cat ซึ่งเป็นระดับความรุนแรงที่ได้จัดไว้ก่อนหน้า

In [ ]:

city = pd.get_dummies(train_data['city'])
X = pd.concat([train_data['longitude'],train_data['latitude'],city], axis = 1)
#X = pd.get_dummies(train_data['city'])
y = train_data[['mag_cat']]

X

Data หลังจากทำ one hot

In [ ]:
#แบ่งข้อมูล test train ใช้อัตราส่วน train 80 test 20 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=44)
print(X.shape)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

จากข้อมูล 13335 แถวทำการแบ่งข้อมูลโดยใช้ train_test_split อัตราส่วน TrainData 80% = 10668แถว (2057, 2) และ TestData 20% = 2667แถว

## เริ่ม Train Model หลายๆแบบ

### Support Vector Machines Model
เลือกใช้ RBF kernel (Radial Basis Function) เนื่องจากลักษณะการกระจายคล้ายกับรัศมีของแผ่นดินไหว
![](https://scikit-learn.org/stable/_images/sphx_glr_plot_svm_kernels_003.png)

In [ ]:
# Support Vector Machines
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)


### Linear SVC

In [ ]:
# Linear SVC
from sklearn import svm
linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
linear_svc_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, y_train) * 100, 2)


## RandomForest Model

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=2)
random_forest.fit(X_train, y_train)
random_forest_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
#f1_random_forest = round(f1_score(y_test, random_forest_pred, average='macro')* 100, 2)

จากการทดลองทำการปรับ parameter แล้วค่า n_estimators=2 เป็นค่าที่ๆดีที่สุด

### KNN (k nearest neighbors) classification

เลือก Model ประเภทนี้มาพิจารณาเพราะพิกัดแผนที่แต่ละจุดนั้นจะเป็น Contour คล้ายภาพ

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_classification_001.png)

In [ ]:
knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
 metric_params=None, n_jobs=1, n_neighbors=5, p=2,)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_test, y_test) * 100, 2)
#f1_knn = round(f1_score(y_test, knn_pred, average='macro')* 100, 2)

### LGBM Classifier

In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(objective='multiclass', random_state=44)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_test)
acc_lgbm = round(lgbm.score(X_train, y_train) * 100, 2)
#f1_lgbm = round(f1_score(y_test, lgbm_pred, average='macro')* 100, 2)

### สรุปเปรียบเทียบ Accuracy Socre ของ Model แต่ละแบบ

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 
              'Random Forest', 'LightGBM', 'Linear SVC'],
    'Accuracy': [acc_svc, acc_knn,
              acc_random_forest, acc_lgbm, acc_linear_svc],
    #'F1': [f1_svc, f1_knn,f1_random_forest, f1_lgbm, f1_linear_svc]
})
models.sort_values(by='Accuracy', ascending=False)

ทำการหาค่าใน confusion matrix

Random Forest

In [ ]:
confusion_matrix(y_test,random_forest_pred)

Linear SVC

In [ ]:
confusion_matrix(y_test,linear_svc_pred)

LightGBM

In [ ]:
confusion_matrix(y_test, lgbm_pred)

Support Vector Machines

In [ ]:
confusion_matrix(y_test, svc_pred)

KNN

In [ ]:
confusion_matrix(y_test, knn_pred)

จากคะแนนจากแต่ละ Model ค่อนข้างใกล้เคียงกัน
* เลือก Random Forest เป็น Model ในการแสดงผล Model Output เพราะได้ Accuracy สูงสุดในกลุ่ม
* SVM ผลลัพท์ที่ได้จาก Model เป็นค่าเดียวกันทั้งหมด (Mag_cat = 1) อาจจะเป็นเพราะจำนวนของ train_data ช่วง Mag_cat = 1 มีเยอะดัง Histrogram และมี std น้อย
* จาก confusion matrix จะเห็นได้ว่า Random forest ดีที่สุดในกลุ่ม
* แต่โมเดลนี้ก็ยังไม่โมเดลที่ดี ที่จะนำไปใช้งานได้อย่างมีประสิทธิภาพ

### Model Output

นำ test_Data มารันใน Model Random Forest และได้ output ใส่ dataframe final และทำการแปลงระดับความแรง Mag_rate ให้เป็นตัวหนังสือเพื่อให้ง่ายต่อการอ่าน

In [ ]:
final = X_test
final['Mag_cat']  = random_forest_pred
final['Marker_color'] = random_forest_pred

# final = pd.DataFrame({
#         "Latitude": X_test["latitude"],
#         "Longitude": X_test["longitude"],
#         "Mag_cat": y_pred_select,
#         "Marker_color": y_pred_select
#     })
# Map ตัวเลขเป็น Text
final["Mag_rate"] = final["Mag_cat"].replace({"1":"Light", "2": "Moderate", "3": "Strong", "4": "Major", "5": "Great"})
# ทำสีเพื่อแสดงใน visualization
final["Marker_color"] = final["Mag_cat"].replace({"1":"yellow", "2": "orange", "3": "red", "4": "purple", "5": "black"})
#เรียงจากมากไปน้อย
final.sort_values('Mag_rate', ascending=0).head(5000)

จากการรัน Model จะได้ข้อมูลค่าพิกัดต่างๆจาก Test_data ที่รันผ่าน Model แล้วจะได้ข้อมูลออกมาที่ Mag_rate ว่าจุดนั้นมีระดับการเกิดแผ่นดินไหวแรงในระดับใด

### นำผลจากการรัน Model มาแสดงในแผนที่

In [ ]:
map_hooray3 = folium.Map(location=[38.495035, 137.026582],
                    zoom_start = 4) 
## เพิ่มวงกลมสีในแผนที่
for idx, e in final.iterrows():
    Circle(location=[e['latitude'], e['longitude']], radius=1500, color=e['Marker_color']).add_to(map_hooray3)

map_hooray3

จากแผนที่วงกลมสีเหลืองจะเป็นพื้นที่ๆอยู่ในระดับ 1 Light จะมีสีส้มและแดงบ้างที่เป็น Moderate และ Strong

สรุป Model นี้สามารถนำไปใช้งานในเบื้องต้นได้ แต่ยังไม่ใช่ model ที่ดีเพราะ data ข้อมูลแผ่นดินไหวอันนี้และจากการหาเพิ่มเติมนั้นมีมิติที่น้อยและค่าของความสั่นสะเทือนนั้นโดนตัดมาแล้วค่าเริ่มที่ 4.5 และการกระจายตัวนั้นอยู่ที่แถวๆ 4.8 ทำให้หลายๆโมเดลทำนายค่าออกมาเป็นระดับ [mag_cat]=1 ทั้งหมด อาจจะต้องหาปัจจัยอื่นๆมาเพิ่ม เพื่อให้เกิดความแม่นยำมากขึ้นและสามารถนำไปใช้งานจริง เรื่องของการเกิดแผ่นดินไหวอาจจะไม่ใช่ภัยอันตรายทั้งหมด ต้องคิดรวมไปถึงปัจจัยอื่นๆนอกเหนือจากนี้เช่นแผ่นดินไหวที่เกิดขึ้นในทะเลอาจจะมีความสี่ยงที่จะเกิดซึนามิได้